# Debugging, Testing, Optimization

First, some comments on software design:
* Two simple general principles:

(1) KIS: Keep it simple.

(2) DRY: Don't repeat yourself.

* Iterative development (agile development):

(1) General idea: one cannot anticipate every detail of a complex problem.

(2) Start simple (with something that works), then refine/improve.

(3) Identify emerging patterns and structures and continuously adapt the structure of the code. This is called refactoring. For this to work, you will need unit tests (we will see this later).

## Reprise: Implementing modular behavior - the strategy pattern

First, a naive (and not so extensible/modular implementation)

In [ ]:
class Duck:
    def __init__(self):
        pass
    def quack(self):
        print("Quack!")
    def display(self):
        print("Standard duck.")
    def take_off(self):
        print("Running fast and flapping ...")
    def fly_to(self, where):
        "No flying to {}".format(where)
    def land(self):
        print("Slowing down, touch down ...")

class RedHeadDuck(Duck):
    def display(self):
        print("Duck with a red head.")

class RubberDuck(Duck):
    def quack(self):
        print("Squeak!")
    def display(self):
        print("Small yellow rubber duck.")

Now, what to do here - for the RubberDuck we would actually need to implement/override all flying related methods.

What if an instance of a "normal" duck is injured and has a broken wing?

How can we adapt for that?

Idea ==> Create a FlyingBehaviorClass which can be plugged into the Duck class.

In [ ]:
class FlyingBehavior:
    def take_off(self):
        print("Running fast and flapping ...")
    def fly_to(self, where):
        "No flying to {}".format(where)
    def land(self):
        print("Slowing down, touch down ...")
        
class Duck:
    def __init__(self):
        self.flying_behavior = FlyingBehavior()
    def take_off(self):
        self.flying_behavior.take_off()
    def fly_to(self, where):
        self.flying_behavior.fly_to(where)
    def land(self):
        self.flying_behavior.land()
        
class NonFlyingBehavior(FlyingBehavior):
    def take_off(self):
        print("This is not working (not implemented)!")
    def fly_to(self, where):
        raise Exception("Not flying ...")
    def land(self):
        print("This will not be necessary ...")
        
class RubberDuck(Duck):
    def __init__(self):
        self.flying_behavior = NonFlyingBehavior()
    def quack(self):
        print("Squeak!")
    def display(self):
        print("Small yellow rubber duck.")
        
class DecoyDuck(Duck):
    def __init__(self):
        self.flying_behavior = NonFlyingBehavior()


## Analysis
* If a Duck breaks a wing, then we can replace the flying strategy by a NonFlyingBehavior instance
* The example relies less on inheritance but on composition
* The behavior class (and subclasses), implementing the strategy allow for dynamic tuning of the behavior of the Duck objects ... (!)
* This "dynamic tuning" is also possible at runtime (!!)
* Strategy pattern in a nutshell: (1) Encapsulate the different strategies in different classes. (2) Store a strategy object in your main object as an attribute. (3) Delegate all strategy calls (the method calls) to the strategy object - see the Duck class and the FlyingBehavior class in the example.

## Debugging: The process of removing errors from your code
* Syntax errors
* Exceptions
* Logic errors

The latter two are usually the most difficult ones (while logic errors are typically the most difficult ones).

### How to test a function

* Try out different combinations of (valid) inputs
* However: Depending on how our program is using it, there could be weird cases, e.g., sort([]), sort([2]).
* These are called edge cases  - the program should work, but it was probably not necessarily what you had in mind. So for functions, we want to test several general cases and several edge cases, and make sure that if something violates it is precondition it reacts appropriately.

In [5]:
def longestSubSeq(myList):
    lastItem = myList[0]
    current = 0
    best = 0
    for item in myList:
        if(item == lastItem):
            best = best + 1
            if best < current:
                best = current
        else:
            lastItem = item
            current = 1
    return best

Now, we need some good test cases:

In [6]:
def test():
    print(longestSubSeq([1]))
    print(longestSubSeq([1, 2, 3]))
    print(longestSubSeq([1, 1, 1, 2]))
    print(longestSubSeq([1, 1, 2, 1]))
    print(longestSubSeq([1, 1, 2, 3, 2, 2, 2, 2]))

test()

1
1
3
2
5


Now, how do we find the error (You will notice, that actually the last test fails - please verify ...)
One very simple technique is using print statements.

This is something you will try out as an *exercise* - for fixing the example (above).

Essentially, the test function should print the sequence: 1 1 3 2 4 (!)

## Test driven development - write unit cases for testing your code

Examples:

In [7]:
import unittest

def hello_world():
    pass

class MyFirstUnitTest(unittest.TestCase):
    def test_hello_world(self):
        self.assertEqual(hello_world(), 'hello world')
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

F
FAIL: test_hello_world (__main__.MyFirstUnitTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-7-13f76e73190c>", line 8, in test_hello_world
    self.assertEqual(hello_world(), 'hello world')
AssertionError: None != 'hello world'

----------------------------------------------------------------------
Ran 1 test in 0.003s

FAILED (failures=1)


Now, of course the test should not fail but pass.
This is what you will look into as an *exercise* - how to fix the code above.

More on unit tests: https://docs.python.org/3/library/unittest.html

## Important comment:
"Program testing can be used to show the presence of bugs, but never to show their absence" (Edsger Dijkstra, 1970)

## Optimization & the need for speed

Beware: "We should forget about small efficiencies, say about 97% of the time: Premature optimization is the root of all evil." (Donald Knuth)

But: When your programming is done (and you have tests (!)) you can optimize the "hotspots".

How to speed up your code:
* Compiling code:  Cython, an optimizing static compiler as well as a compiled language which generates Python modules that can be used by regular Python code.
* Simple alternative: Numba, a Numpy-aware optimizing just-in-time compiler.

### Numba
Numba provides a Just-In-Time compiler for Python code. Just-in-time compilation refers to the process of compiling during execution rather than before-hand. It uses the LLVM infrastructure to compile Python code into machine code. 
Central to the use of Numba is the numba.jit decorator.

In [15]:
import numba

@numba.jit
def f(x):
    return x**2-x

def integrate_f(a, b, N):
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

%time integrate_f(1, 100, 100000)

CPU times: user 58.4 ms, sys: 2.66 ms, total: 61 ms
Wall time: 60.7 ms


328328.5995161719

In [16]:
import numba 
from numba import float64, int32

@numba.jit
def f(x):
    return x**2-x

@numba.jit(float64(float64, float64, int32))
def integrate_f(a, b, N):
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

%time integrate_f(1, 100, 100000)

CPU times: user 115 µs, sys: 2 µs, total: 117 µs
Wall time: 120 µs


328328.5995161719

So, with the second (optimized) example, you should see a very large speed improvement (!) :-)